In [4]:
! pip install kaggle
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (12.6 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debco

Libraries Installation. Google Colab may encounter issues & not properly include Kaggle from the start. We have to install pytesseract and properly link it to our project.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Link your Google Drive to the Colab

In [6]:
! mkdir ~/.kaggle

In [7]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

Copy your Kaggle credentials, they are stored inside the kaggle.json

In [8]:
! chmod 600 ~/.kaggle/kaggle.json

In [9]:
import os

folder_path = "/content/drive/MyDrive/car-plate-detection"

if not os.path.exists(folder_path):
    !kaggle datasets download andrewmvd/car-plate-detection
    !mkdir "/content/drive/MyDrive/car-plate-detection"
    !unzip "/content/car-plate-detection.zip" -d "/content/drive/MyDrive/car-plate-detection"

Download the dataset & de-compress it

In [11]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet

from glob import glob
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

Import every library we will use for the model construction & training.
- Glob processes large chunks of raw data
- skimage processes pictures
- tensorflow trains the Model
- matplotlib displays results
- pandas & numpy handle the filepaths & characteristics

In [12]:
path = glob('/content/drive/MyDrive/car-plate-detection/annotations/*.xml')
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

Read, process & extract the annotated Bounding Boxes information from the XML files

In [13]:
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,/content/drive/MyDrive/car-plate-detection/ann...,134,262,128,160
1,/content/drive/MyDrive/car-plate-detection/ann...,226,419,125,173
2,/content/drive/MyDrive/car-plate-detection/ann...,195,244,266,282
3,/content/drive/MyDrive/car-plate-detection/ann...,230,248,129,134
4,/content/drive/MyDrive/car-plate-detection/ann...,138,177,79,92


Sample output of the retrieved data

In [69]:
from PIL import Image
import requests
url = 'https://images.bfmtv.com/P-SjIS8mmRWOWD1ZrV46gt7j37w=/6x4:1254x706/800x0/images/-158334.jpg'
im = Image.open(requests.get(url, stream=True).raw)
im.save('/content/carTest.jpg')

save test picture to local folder

In [14]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('/content/drive/MyDrive/car-plate-detection/images/',filename_image)
    return filepath_image

Properly format the filepath

In [15]:
image_path = list(df['filepath'].apply(getFilename))
file_path = image_path[87]
img = cv2.imread(file_path) #read the image
img = io.imread(file_path) #Read the image
fig = px.imshow(img)
fig.update_layout(width=600, height=500, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Cars333.png with bounding box')
fig.add_shape(type='rect',x0=df['xmin'][87], x1=df['xmax'][87], y0=df['ymin'][87], y1=df['ymax'][87], xref='x', yref='y',line_color='cyan')

Example of a Car picture & its License Plate Bounding box

In [16]:
#Targeting all our values in array selecting all columns
labels = df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # Prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output
    # Append
    data.append(norm_load_image_arr)
    output.append(label_norm)

Iterate through each picture to associate them with their bounding boxes coordinates

In [17]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

Convert data to array

In [18]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((346, 224, 224, 3), (87, 224, 224, 3), (346, 4), (87, 4))

Split the data into training and testing set using sklearn.


In [19]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False, input_tensor=Input(shape=(224,224,3)))
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)


# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

219055592/219055592 [==============================] - 1s 0us/step


construct the model

In [20]:
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalization[0][0

# Complie model


In [21]:
tfb = TensorBoard('object_detection')
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=80,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/80
35/35 [==============================] - 117s 517ms/step - loss: 0.0394 - val_loss: 0.0279
Epoch 2/80
35/35 [==============================] - 8s 226ms/step - loss: 0.0109 - val_loss: 0.0210
Epoch 3/80
35/35 [==============================] - 8s 228ms/step - loss: 0.0069 - val_loss: 0.0177
Epoch 4/80
35/35 [==============================] - 8s 228ms/step - loss: 0.0054 - val_loss: 0.0169
Epoch 5/80
35/35 [==============================] - 8s 225ms/step - loss: 0.0045 - val_loss: 0.0209
Epoch 6/80
35/35 [==============================] - 8s 232ms/step - loss: 0.0041 - val_loss: 0.0253
Epoch 7/80
35/35 [==============================] - 8s 227ms/step - loss: 0.0040 - val_loss: 0.0202
Epoch 8/80
35/35 [==============================] - 8s 232ms/step - loss: 0.0032 - val_loss: 0.0155
Epoch 9/80
35/35 [==============================] - 8s 231ms/step - loss: 0.0027 - val_loss: 0.0131
Epoch 10/80
35/35 [==============================] - 8s 236ms/step - loss: 0.0026 - val_loss: 0.01

Train model

In [22]:
model.save('/content/drive/MyDrive/car-plate-detection/object_detection.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



Save model weights locally to ensure they're properly constructed. The ".keras" extension is more modern but caused issues during testing, hence the reversion to the ".h5" extension.

In [23]:
# Load model
model = tf.keras.models.load_model('/content/drive/MyDrive/car-plate-detection/object_detection.h5')
print('Model loaded Sucessfully')

Model loaded Sucessfully


In [24]:
path = '/content/drive/MyDrive/car-plate-detection/images/Cars0.png'
image = load_img(path) # PIL object
image = np.array(image,dtype=np.uint8) # 8 bit array (0,255)
image1 = load_img(path,target_size=(224,224))
image_arr_224 = img_to_array(image1)/255.0  # Convert into array and get the normalized output

# Size of the orginal image
h,w,d = image.shape
print('Height of the image =',h)
print('Width of the image =',w)


Height of the image = 268
Width of the image = 500


Show a test picture

In [25]:
fig = px.imshow(image)
fig.update_layout(width=700, height=500,  margin=dict(l=10, r=10, b=10, t=10), xaxis_title='TEST Image')

In [26]:
test_arr = image_arr_224.reshape(1,224,224,3)
coords = model.predict(test_arr)
denorm = np.array([w,w,h,h])
coords = coords * denorm

1/1 [==============================] - 5s 5s/step


Begin Prediction

In [27]:
coords = coords.astype(np.int32)
xmin, xmax,ymin,ymax = coords[0]
pt1 =(xmin,ymin)
pt2 =(xmax,ymax)
print(pt1, pt2)

(235, 127) (431, 169)


In [28]:
cv2.rectangle(image,pt1,pt2,(0,255,0),3)
fig = px.imshow(image)
fig.update_layout(width=700, height=500, margin=dict(l=10, r=10, b=10, t=10),  xaxis_title="test picture")

We gave it a picture it knows to test whether or not the model could display correctly the Bounding Box

In [70]:
# Create pipeline
path = '/content/carTest.jpg'

def object_detection(path):
    # Read image
    image = load_img(path) # PIL object
    image = np.array(image,dtype=np.uint8) # 8 bit array (0,255)
    image1 = load_img(path,target_size=(224,224))

    # Data preprocessing
    image_arr_224 = img_to_array(image1)/255.0 # Convert to array & normalized
    h,w,d = image.shape
    test_arr = image_arr_224.reshape(1,224,224,3)

    # Make predictions
    coords = model.predict(test_arr)

    # Denormalize the values
    denorm = np.array([w,w,h,h])
    coords = coords * denorm
    coords = coords.astype(np.int32)

    # Draw bounding on top the image
    xmin, xmax,ymin,ymax = coords[0]
    pt1 =(xmin,ymin)
    pt2 =(xmax,ymax)
    print(pt1, pt2)
    cv2.rectangle(image,pt1,pt2,(0,255,0),3)
    return image, coords

image, cods = object_detection(path)

fig = px.imshow(image)
fig.update_layout(width=700, height=500, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Bounding Box Prediction')

1/1 [==============================] - 0s 39ms/step
(316, 249) (498, 306)


In [74]:
img = np.array(load_img(path))
xmin ,xmax,ymin,ymax = cods[0]
roi = img[ymin:ymax,xmin:xmax]
fig = px.imshow(roi)
fig.update_layout(width=(xmax-xmin), height=(ymax-ymin), margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Cropped image')

Extraction of the license plates with the coordinates the model found

In [75]:
# extract text from image
text = pt.image_to_string(roi)
print(text)

B 2228 HN



Extract text from the picture

In [76]:
from sklearn.metrics import f1_score, precision_score, recall_score

Performance evaluation with measurements of the f1_score

In [78]:
from sklearn.metrics import f1_score, precision_score, recall_score

# Make predictions
y_pred = model.predict(x_test)

3/3 [==============================] - 0s 183ms/step


In [38]:
# Convert predicted coordinates to actual bounding boxes
denorm = np.array([w, w, h, h])
y_pred_denorm = y_pred * denorm
y_pred_denorm = y_pred_denorm.astype(np.int32)

In [39]:
# Convert ground truth coordinates to actual bounding boxes
y_true_denorm = y_test * denorm
y_true_denorm = y_true_denorm.astype(np.int32)

In [40]:

# Flatten the arrays for f1_score calculation
y_true_flat = y_true_denorm.flatten()
y_pred_flat = y_pred_denorm.flatten()

In [41]:
# Apply a threshold (adjust as needed)
threshold = 0.5
y_pred_flat_binary = (y_pred_flat > threshold).astype(int)

In [51]:
# Calculate F1 score, precision, and recall
f1 = f1_score(y_true_flat, y_pred_flat_binary, average='micro')
precision = precision_score(y_true_flat, y_pred_flat_binary, average='micro')
recall = recall_score(y_true_flat, y_pred_flat_binary, average='micro')

In [52]:

print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

F1 Score: 0.0
Precision: 0.0
Recall: 0.0
